# 1. Objective:


A phishing website is a common social engineering method that mimics trustful uniform resource locators (URLs) and webpages. The objective of this notebook is to collect data & extract the selctive features form the URLs.**

# **Importing Libraries**

In [ ]:
import pandas as pd
import ipaddress
import re
!pip install python-whois
import urllib
import urllib.request
import requests
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import keras
import tensorflow as tf
from keras.models import Model
from sklearn import metrics
from sklearn.svm import SVC
from urllib.parse import urlparse,urlencode
from bs4 import BeautifulSoup
from datetime import datetime
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from keras.layers import Input, Dense
from keras import regularizers

# **2. Collecting Data from csv file**

For this particular project i collected a bunch URLs bifarcating into legitimate and phising

The collection of phishing Urls are from phishtank.com meanwhile for the legitimate Urls are been collected from unb.ca

# **2.1 Phishing Urls**

In [ ]:
#Downloading the phishing URLs file
!wget http://data.phishtank.com/data/online-valid.csv

URL transformed to HTTPS due to an HSTS policy
--2025-04-10 07:54:48--  https://data.phishtank.com/data/online-valid.csv
Resolving data.phishtank.com (data.phishtank.com)... 104.18.161.237, 104.18.162.237, 2606:4700::6812:a2ed, ...
Connecting to data.phishtank.com (data.phishtank.com)|104.18.161.237|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn.phishtank.com/datadumps/verified_online.csv?Expires=1744271561&Signature=Ki6eust8~nKT-gr9QAf1WdH1nR1sdcIAyarD9w5F0obwJqU02qayCGeeFQ2qxhf~3kNEHgkj7p6NkdaqBlBcKJhjGQ4MxcQscE41Bm2xDLL6EEEQurzoESOXmHDzBiHOWLmAw6hcjD3e1CUoNeYrfm47MyO-luOqUO5HZ36cnloVzKxZjdQ3zQzGRrtTwIZX4phdx8-ljW0W7aq4yop8cf0oYyUCQuvnIoM7JsWqTAt9gFqgcSP7tO6pcEFkXe-Ld0ouMu3qJh1pOygHZvil1s042arboGcXcRKacO7bUxDjacEefrpesT1QnlGPU8AYVDNoK~MllrCbchpHhd47Cg__&Key-Pair-Id=APKAILB45UG3RB4CSOJA [following]
--2025-04-10 07:54:48--  https://cdn.phishtank.com/datadumps/verified_online.csv?Expires=1744271561&Signature=Ki6eust8~nKT-gr9QAf1WdH1nR1sdcIAyar

In [ ]:
#loading the phishing URLs data to dataframe
data0 = pd.read_csv("online-valid.csv")
data0.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,9054831,https://ezpass.com-ezbqo.vip/,http://www.phishtank.com/phish_detail.php?phis...,2025-04-10T05:51:59+00:00,yes,2025-04-10T07:12:20+00:00,yes,Other
1,9054830,https://ezpass.com-ezbqi.vip/,http://www.phishtank.com/phish_detail.php?phis...,2025-04-10T05:51:47+00:00,yes,2025-04-10T07:12:20+00:00,yes,Other
2,9054829,https://ezpass.com-ezbqd.vip/,http://www.phishtank.com/phish_detail.php?phis...,2025-04-10T05:51:35+00:00,yes,2025-04-10T07:12:20+00:00,yes,Other
3,9054828,https://ezpass.com-ezbqa.vip/,http://www.phishtank.com/phish_detail.php?phis...,2025-04-10T05:51:19+00:00,yes,2025-04-10T07:12:20+00:00,yes,Other
4,9054827,https://ezpass.com-euxx.top/,http://www.phishtank.com/phish_detail.php?phis...,2025-04-10T05:51:07+00:00,yes,2025-04-10T07:12:20+00:00,yes,Other


In [ ]:
data0.shape

(64066, 8)

In [ ]:
#Collecting 5,000 Phishing URLs randomly
phishurl = data0.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6514946,http://confirmprofileaccount.com/,http://www.phishtank.com/phish_detail.php?phis...,2020-04-19T11:06:55+00:00,yes,2020-04-19T13:42:41+00:00,yes,Other
1,4927651,http://www.marreme.com/MasterAdmin/04mop.html,http://www.phishtank.com/phish_detail.php?phis...,2017-04-04T19:35:54+00:00,yes,2017-05-03T23:00:42+00:00,yes,Other
2,5116976,http://modsecpaststudents.com/review/,http://www.phishtank.com/phish_detail.php?phis...,2017-07-25T18:48:30+00:00,yes,2017-07-28T16:01:36+00:00,yes,Other
3,6356131,https://docs.google.com/forms/d/e/1FAIpQLScL6L...,http://www.phishtank.com/phish_detail.php?phis...,2020-01-13T20:13:37+00:00,yes,2020-01-17T01:55:38+00:00,yes,Other
4,6535965,https://oportunidadedasemana.com/americanas//?...,http://www.phishtank.com/phish_detail.php?phis...,2020-04-29T00:01:03+00:00,yes,2020-05-01T10:55:35+00:00,yes,Other


In [ ]:
phishurl.shape

(5000, 8)

# 2.2 Legitimate URLs

In [ ]:
#Loading legitimate files
data1 = pd.read_csv("Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Benign_list_big_final.csv'

In [ ]:
#Collecting 5,000 Legitimate URLs randomly
legiurl = data1.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

In [ ]:
legiurl.shape

# 3. Feature Extraction:
In this step, features are extracted from the URLs dataset.

The extracted features are categorized into

Address Bar based Features
Domain based Features
HTML & Javascript based Features

# **3.1. Address Bar Based Features:**
Many features can be extracted that can be consided as address bar base features. Out of them, below mentioned were considered for this project.


1. Domain of URL
2. IP Address in URL
3. "@" Symbol in URL
4. Length of URL
5. Depth of URL
6. Redirection "//" in URL
7. "http/https" in Domain name
8. Using URL Shortening Services “TinyURL”
9. Prefix or Suffix "-" in Domain





Each of these features are explained and coded below:

In [ ]:
# 1.Domain of the URL (Domain)
def getDomain(url):
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

In [ ]:
# 2.Checks for IP address in URL (Have_IP)
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

In [ ]:
# 3.Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    at = 1
  else:
    at = 0
  return at

In [ ]:
# 4.Finding the length of URL and categorizing (URL_Length)
def getLength(url):
  if len(url) < 54:
    length = 0
  else:
    length = 1
  return length

In [ ]:
# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [ ]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [ ]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [ ]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [ ]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [ ]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

# **3.2. Domain Based Features:**
Many features can be extracted that come under this category. Out of them, below mentioned were considered for this project.

DNS Record
Website Traffic
Age of Domain
End Period of Domain
Each of these features are explained and the coded below:

In [ ]:
# 10.Web traffic (Web_Traffic)
def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

In [ ]:
# 11.Survival time of domain: The difference between termination time and creation time (Domain_Age)
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

In [ ]:
# 12.End time of domain: The difference between termination time and current time (Domain_End)
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

# **3.3. HTML and JavaScript based Features**
Many features can be extracted that come under this category. Out of them, below mentioned were considered for this project.

1. IFrame Redirection
2. Status Bar Customization
3. Disabling Right Click
4. Website Forwarding
Each of these features are explained and the coded below:

In [ ]:
# 13. IFrame Redirection (iFrame)
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0
      else:
          return 1

In [ ]:
# 14.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response):
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

In [ ]:
# 15.Checks the status of the right click attribute (Right_Click)
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [ ]:
# 16.Checks the number of forwardings (Web_Forwards)
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1


In [ ]:
#Function to extract features
def featureExtraction(url,label):

  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))

  #Domain based features (4)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1

  features.append(dns)
  features.append(web_traffic(url))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainEnd(domain_name))

  # HTML & Javascript based features (4)
  try:
    response = requests.get(url)
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)

  return features

# **4.1. Legitimate URLs:**
Now, feature extraction will be done on legitimate URLs.

In [ ]:
legiurl.shape

In [ ]:
# 10.Web traffic (Web_Traffic)
def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    # Check network connectivity before making request
    import socket
    socket.getaddrinfo('data.alexa.com', 80)

    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except (TypeError, socket.gaierror, urllib.error.URLError):
        # Return 1 for any error, indicating web traffic is likely low or unavailable
        return 1
  if rank <100000:
    return 1
  else:
    return 0

# **4.2. Phishing URLs:**
Now, feature extraction is performed on phishing URLs.

In [ ]:
import pandas as pd

# Assuming 'online-valid.csv' contains the phishing URLs data
phishurl = pd.read_csv("online-valid.csv")

In [ ]:
import whois
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import ipaddress
import urllib
from datetime import datetime

# ... (Other functions like getDomain, havingIP, etc. should be defined before this)

def featureExtraction(url, label):
    features = []
    # Address

# **5. Final Dataset**
In the above section we formed two dataframes of legitimate & phishing URL features. Now, we will combine them to a single dataframe and export the data to csv file for the Machine Learning training.

In [ ]:
#Loading legitimate files
data1 = pd.read_csv("Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()

#Collecting 5,000 Legitimate URLs randomly
legiurl = data1.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

import pandas as pd
import whois
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup


In [ ]:
#Loading legitimate files
data1 = pd.read_csv("Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()

#Collecting 5,000 Legitimate URLs randomly
legiurl = data1.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

import pandas as pd
import whois
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup

# Assuming you want to combine phishing and legitimate data into 'urldata'
# Replace 'phishing.csv' and 'legitimate.csv' with the actual filenames if different
phishing_data = pd.read_csv('phishing.csv')
legitimate_data = pd.read_csv('legitimate.csv')

# Concatenate the DataFrames
urldata = pd.concat([legitimate_data, phishing_data])

urldata.tail() # Now urldata is defined and should work

In [ ]:
urldata.shape

In [ ]:
# Storing the data in CSV file
urldata.to_csv('urldata.csv', index=False)

# **6. Data Cleaning and Splitting**
We finally extracted 16 features for 10,000 URL which has 5000 phishing & 5000 legitimate URLs Both phishing and benign URLs of websites are gathered to form a dataset and from them required URL and website content-based features are extracted. The performance level of each model is measures and compared.

In [ ]:
#Loading the data
data0 = pd.read_csv('urldata.csv')
data0.head()

# **6.1 Familiarizing with Data**
In this step, few dataframe methods are used to look into the data and its features.

In [ ]:
#Checking the shape of the dataset
data0.shape

In [ ]:
#Listing the features of the dataset
data0.columns

In [ ]:
#Information about the dataset
data0.info()

# **6.2 Visualizing the data**
Few plots and graphs are displayed to find how the data is distributed and the how features are related to each other.

In [ ]:
# Import necessary libraries
import pandas as pd
import ipaddress
import re
import whois  # Assuming 'whois' is the intended library
import urllib
import urllib.request
import requests
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt  # Import matplotlib.pyplot as plt
import pickle
import keras
import tensorflow as tf
from keras.models import Model
from sklearn import metrics
from sklearn.svm import SVC
from urllib.parse import urlparse, urlencode
from bs4 import BeautifulSoup
from datetime import datetime
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from keras.layers import Input, Dense
from keras import regularizers

# ... (rest of your code) ...

# Plotting the data distribution
data0.hist(bins=50, figsize=(15, 15))
plt.show()


# **6.3 Data Preprocessing**
Here, we clean the data by applying data preprocesssing techniques and transform the data to use it in the models.

In [ ]:
data0.describe()

In [ ]:
#Dropping the Domain column
data = data0.drop(['Domain'], axis = 1).copy()

In [ ]:
#checking the data for null or missing values
data.isnull().sum()

In [ ]:
# shuffling the rows in the dataset so that when splitting the train and test set are equally distributed
data = data.sample(frac=1).reset_index(drop=True)
data.head()


# **6.4 Splitting the Data**

In [ ]:
# Sepratating & assigning features and target columns to X & y
y = data['Label']
X = data.drop('Label',axis=1)
X.shape, y.shape

In [ ]:
# Splitting the dataset into train and test sets: 80-20 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2, random_state = 12)
X_train.shape, X_test.shape

# **7. Comparision and Training of Dataset using Different Supervised Machine Learning Algorithmn**

From the dataset above, it is clear that this is a supervised machine learning task. There are two major types of supervised machine learning problems, called classification and regression.

This data set comes under classification problem, as the input URL is classified as phishing or legitimate. The supervised machine learning models (classification) considered to train the dataset in this notebook are:

1. Decision Tree
2. Random Forest
3. Multilayer Perceptrons
4. XGBoost
5. Autoencoder Neural Network
6. Support Vector Machines

In [ ]:
# Creating holders to store the model performance results
ML_Model = []
acc_train = []
acc_test = []

#function to call for storing the results
def storeResults(model, a,b):
  ML_Model.append(model)
  acc_train.append(round(a, 3))
  acc_test.append(round(b, 3))

# **7.1. Decision Tree Classifier**
Decision trees are widely used models for classification and regression tasks. Essentially, they learn a hierarchy of if/else questions, leading to a decision. Learning a decision tree means learning the sequence of if/else questions that gets us to the true answer most quickly.

In the machine learning setting, these questions are called tests (not to be confused with the test set, which is the data we use to test to see how generalizable our model is). To build a tree, the algorithm searches over all possible tests and finds the one that is most informative about the target variable.

In [ ]:
# Decision Tree model
from sklearn.tree import DecisionTreeClassifier

# instantiate the model
tree = DecisionTreeClassifier(max_depth = 5)
# fit the model
tree.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_tree = tree.predict(X_test)
y_train_tree = tree.predict(X_train)

# **Performance Evalution**

In [ ]:
#computing the accuracy of the model performance
acc_train_tree = accuracy_score(y_train,y_train_tree)
acc_test_tree = accuracy_score(y_test,y_test_tree)

print("Decision Tree: Accuracy on training Data: {:.3f}".format(acc_train_tree))
print("Decision Tree: Accuracy on test Data: {:.3f}".format(acc_test_tree))

In [ ]:
#checking the feature importance in the model
plt.figure(figsize=(9,7))
n_features = X_train.shape[1]
plt.barh(range(n_features), tree.feature_importances_, align='center')
plt.yticks(np.arange(n_features), X_train.columns)
plt.xlabel("Feature importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Decision Tree', acc_train_tree, acc_test_tree)

# **7.2. Random Forest Classifier**
Random forests for regression and classification are currently among the most widely used machine learning methods.A random forest is essentially a collection of decision trees, where each tree is slightly different from the others. The idea behind random forests is that each tree might do a relatively good job of predicting, but will likely overfit on part of the data.

If we build many trees, all of which work well and overfit in different ways, we can reduce the amount of overfitting by averaging their results. To build a random forest model, you need to decide on the number of trees to build (the n_estimators parameter of RandomForestRegressor or RandomForestClassifier). They are very powerful, often work well without heavy tuning of the parameters, and don’t require scaling of the data.

In [ ]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier

# instantiate the model
forest = RandomForestClassifier(max_depth=5)

# fit the model
forest.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_forest = forest.predict(X_test)
y_train_forest = forest.predict(X_train)

# ***Performance Evaluation***

In [ ]:
#computing the accuracy of the model performance
acc_train_forest = accuracy_score(y_train,y_train_forest)
acc_test_forest = accuracy_score(y_test,y_test_forest)

print("Random forest: Accuracy on training Data: {:.3f}".format(acc_train_forest))
print("Random forest: Accuracy on test Data: {:.3f}".format(acc_test_forest))

In [ ]:
#checking the feature importance in the model
plt.figure(figsize=(9,7))
n_features = X_train.shape[1]
plt.barh(range(n_features), forest.feature_importances_, align='center')
plt.yticks(np.arange(n_features), X_train.columns)
plt.xlabel("Feature importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Random Forest', acc_train_forest, acc_test_forest)

# **7.3. Multilayer Perceptrons (MLPs): Deep Learning**
Multilayer perceptrons (MLPs) are also known as (vanilla) feed-forward neural networks, or sometimes just neural networks. Multilayer perceptrons can be applied for both classification and regression problems.

MLPs can be viewed as generalizations of linear models that perform multiple stages of processing to come to a decision.

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

# ... (Your previous code to load and prepare data) ...

# Sepratating & assigning features and target columns to X & y
y = data['Label']
X = data.drop('Label', axis=1)

# Splitting the dataset into train and test sets: 80-20 split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=12)

# Impute missing values using the mean
imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent'
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)  # Use the same imputer fitted on training data

# instantiate the model
mlp = MLPClassifier(alpha=0.001, hidden_layer_sizes=([100, 100, 100]))

# fit the model
mlp.fit(X_train, y_train)

# ... (Rest of your code) ...

In [ ]:
# Multilayer Perceptrons model
from sklearn.neural_network import MLPClassifier

# instantiate the model
mlp = MLPClassifier(alpha=0.001, hidden_layer_sizes=([100,100,100]))

# fit the model
mlp.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_mlp = mlp.predict(X_test)
y_train_mlp = mlp.predict(X_train)

# **7.4. XGBoost Classifier**
XGBoost is one of the most popular machine learning algorithms these days. XGBoost stands for eXtreme Gradient Boosting. Regardless of the type of prediction task at hand; regression or classification. XGBoost is an implementation of gradient boosted decision trees designed for speed and performance.

In [ ]:
# instantiate the model
xgb = XGBClassifier(learning_rate=0.4,max_depth=7)
#fit the model
xgb.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_xgb = xgb.predict(X_test)
y_train_xgb = xgb.predict(X_train)

# **Performance Evalution**

In [ ]:
#computing the accuracy of the model performance
acc_train_xgb = accuracy_score(y_train,y_train_xgb)
acc_test_xgb = accuracy_score(y_test,y_test_xgb)

print("XGBoost: Accuracy on training Data: {:.3f}".format(acc_train_xgb))
print("XGBoost : Accuracy on test Data: {:.3f}".format(acc_test_xgb))

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('XGBoost', acc_train_xgb, acc_test_xgb)

# **7.5. Autoencoder Neural Network**
An auto encoder is a neural network that has the same number of input neurons as it does outputs. The hidden layers of the neural network will have fewer neurons than the input/output neurons. Because there are fewer neurons, the auto-encoder must learn to encode the input to the fewer hidden neurons. The predictors (x) and output (y) are exactly the same in an auto encoder.

In [ ]:
#building autoencoder model

input_dim = X_train.shape[1]
encoding_dim = input_dim

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="relu",
                activity_regularizer=regularizers.l1(10e-4))(input_layer)
encoder = Dense(int(encoding_dim), activation="relu")(encoder)

encoder = Dense(int(encoding_dim-2), activation="relu")(encoder)
code = Dense(int(encoding_dim-4), activation='relu')(encoder)
decoder = Dense(int(encoding_dim-2), activation='relu')(code)

decoder = Dense(int(encoding_dim), activation='relu')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.summary()

In [ ]:
#compiling the model
autoencoder.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

#Training the model
history = autoencoder.fit(X_train, X_train, epochs=10, batch_size=64, shuffle=True, validation_split=0.2)

# ***Performance Evalution***

In [ ]:
acc_train_auto = autoencoder.evaluate(X_train, X_train)[1]
acc_test_auto = autoencoder.evaluate(X_test, X_test)[1]

print('\nAutoencoder: Accuracy on training Data: {:.3f}' .format(acc_train_auto))
print('Autoencoder: Accuracy on test Data: {:.3f}' .format(acc_test_auto))

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('AutoEncoder', acc_train_auto, acc_test_auto)

# **7.6. Support Vector Machines**
In machine learning, support-vector machines (SVMs, also support-vector networks) are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. Given a set of training examples, each marked as belonging to one or the other of two categories, an SVM training algorithm builds a model that assigns new examples to one category or the other, making it a non-probabilistic binary linear classifier.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer # Import SimpleImputer

# Assuming 'data' is your DataFrame containing features and labels
# ... (Your data loading and preprocessing code) ...

# Sepratating & assigning features and target columns to X & y
y = data['Label']
X = data.drop('Label', axis=1)

# Check for unique classes in the target variable
unique_classes = np.unique(y)
print("Unique classes in target variable:", unique_classes)

# If only one class is found, investigate data loading and preprocessing
# Instead of skipping training, print a message and continue
if len(unique_classes) < 2:
    print("Warning: Target variable has only one class. Model performance may be affected.")
    # You might want to add code here to handle this case, such as:
    # - Resampling the data to balance classes
    # - Using a different model that is suitable for single-class classification

# Splitting the dataset into train and test sets: 80-20 split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.2, random_state=12)

# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # Replace 'mean' with other strategies if needed
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Initiate the SVM model
svm = SVC(kernel='linear', C=1.0, random_state=12)

# Fit the model
svm.fit(X_train, y_train)

# ... (Rest of your code) ...

# **Performance Evalution**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer # Import SimpleImputer

# Assuming 'data' is your DataFrame containing features and labels
# ... (Your data loading and preprocessing code) ...

# Sepratating & assigning features and target columns to X & y
y = data['Label']
X = data.drop('Label', axis=1)

# ... (Your code to check unique classes in target variable) ...

# Splitting the dataset into train and test sets: 80-20 split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.2, random_state=12)

# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # Replace 'mean' with other strategies if needed
X_train = imputer.fit_transform(X_train) # Impute on X_train
X_test = imputer.transform(X_test) # Impute on X_test

# Initiate the SVM model
svm = SVC(kernel='linear', C=1.0, random_state=12)

# Fit the model
svm.fit(X_train, y_train)

# ... (Rest of your code) ...

In [ ]:
# Initiate the SVM model
svm = SVC(kernel='linear', C=1.0, random_state=12)

# Fit the model
svm.fit(X_train, y_train)

# Predict on training data
y_train_svm = svm.predict(X_train) #This line is added to calculate y_train_svm

# Predict on test data
y_test_svm = svm.predict(X_test)

#computing the accuracy of the model performance
acc_train_svm = accuracy_score(y_train,y_train_svm)
acc_test_svm = accuracy_score(y_test,y_test_svm)

print("SVM: Accuracy on training Data: {:.3f}".format(acc_train_svm))
print("SVM : Accuracy on test Data: {:.3f}".format(acc_test_svm))

In [ ]:
#computing the accuracy of the model performance
acc_train_svm = accuracy_score(y_train,y_train_svm)
acc_test_svm = accuracy_score(y_test,y_test_svm)

print("SVM: Accuracy on training Data: {:.3f}".format(acc_train_svm))
print("SVM : Accuracy on test Data: {:.3f}".format(acc_test_svm))

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('SVM', acc_train_svm, acc_test_svm)

# **8. Comparision of Models**
To compare the models performance, a dataframe is created. The columns of this dataframe are the lists created to store the results of the model.

In [ ]:
#creating dataframe
results = pd.DataFrame({ 'ML Model': ML_Model,
    'Train Accuracy': acc_train,
    'Test Accuracy': acc_test})
results

# **Saving the Model**

In [ ]:
# save XGBoost model to file
pickle.dump(xgb, open("XGBoostClassifier.pickle.dat", "wb"))